In [ ]:
# general imports
import pickle
import random

from endstate_rew.system import generate_molecule, initialize_simulation
from endstate_rew.neq import perform_switching

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import torch
# Imports from the openff toolkit

from openmm import unit

from pymbar import BAR, EXP
from tqdm import tqdm

num_threads = 4
torch.set_num_threads(num_threads)

# generate a molecule using openff
###################
###################
name = 'acetylacetone'
n_samples = 10#2_000
n_steps_per_sample = 1_000
smiles = 'CC(C(C)=O)C(C)=O'
###################
###################
molecule = generate_molecule(smiles)
sim = initialize_simulation(molecule)

In [ ]:
mm_samples = collect_samples(sim, n_samples=n_samples, n_steps_per_sample=n_steps_per_sample, lamb=0.0)
qml_samples = collect_samples(sim, n_samples=n_samples, n_steps_per_sample=n_steps_per_sample, lamb=1.0)
#pickle.dump(mm_samples, open(f'../data/{name}_mm_samples_{n_samples}_{n_steps_per_sample}.pickle', 'wb+'))
#pickle.dump(qml_samples, open(f'../data/{name}_qml_samples_{n_samples}_{n_steps_per_sample}.pickle', 'wb+'))


In [ ]:
#mm_samples = pickle.load(open(f'../data/{name}_mm_samples.pickle', 'rb'))
#qml_samples = pickle.load(open(f'../data/{name}_qml_samples.pickle', 'rb'))

In [ ]:
# NEQ
switching_length = 1_001
nr_of_switches = 5

lambs = np.linspace(0,1,switching_length)
ws_from_mm_to_qml = perform_switching(sim, lambs, samples=mm_samples,nr_of_switches=nr_of_switches)
#pickle.dump(ws_from_mm_to_qml, open(f'../data/neq_ws_from_mm_to_qml_{name}.pickle', 'wb+'))
lambs = np.linspace(1,0,switching_length)
ws_from_qml_to_mm = perform_switching(sim, lambs, samples=qml_samples,nr_of_switches=nr_of_switches)
#pickle.dump(ws_from_mm_to_qml, open(f'../data/neq_ws_from_qml_to_mm_{name}.pickle', 'wb+'))

In [ ]:
print(f"Crooks' equation: {BAR(ws_from_mm_to_qml, ws_from_qml_to_mm)}")
print(f"Jarzynski's equation: {EXP(ws_from_mm_to_qml)}")

In [ ]:
# instantenious swichting (FEP)
switching_length = 2
nr_of_switches = 500

lambs = np.linspace(0,1,switching_length)
ws_from_mm_to_qml = perform_switching(lambs, samples=mm_samples,nr_of_switches=nr_of_switches)
lambs = np.linspace(1,0,switching_length)
ws_from_qml_to_mm = perform_switching(lambs, samples=qml_samples,nr_of_switches=nr_of_switches)
print(f'FEP: From MM to QML: {EXP(ws_from_mm_to_qml)}')
print(f'FEP: From MM to QML: {EXP(ws_from_qml_to_mm)}')
print(f"BAR: {BAR(ws_from_mm_to_qml, ws_from_qml_to_mm)}")